In [2]:
%pip install \
transformers==4.27.2\
    datasets==2.11.0\
        evaluate==0.4.0\
 rouge_score ==0.1.2\
    loralib==0.1.1\
        peft==0.3.0 -- quiet

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '==0.1.2'


In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM ,AutoTokenizer,GenerationConfig,TrainingArguments,Trainer
import torch 
import time
import evaluate
import pandas as pd
import numpy as np

In [5]:
%pip install -U datasets

  Using cached datasets-2.18.0-py3-none-any.whl.metadata (20 kB)
Using cached datasets-2.18.0-py3-none-any.whl (510 kB)
  Attempting uninstall: datasets
    Found existing installation: datasets 2.11.0
    Uninstalling datasets-2.11.0:
      Successfully uninstalled datasets-2.11.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
from datasets import load_dataset
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [6]:
model_name = 'google/flan-t5-base'
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name,torch_dtype =torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model.safetensors: 100%|██████████| 990M/990M [10:12<00:00, 1.62MB/s] 
c:\Users\vrinda\anaconda3\envs\gpu\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vrinda\.cache\huggingface\hub\models--google--flan-t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [7]:
def print_no_of_trainable_model_parameters(model):
    trainable_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    percentage_trainable = (trainable_params / all_model_params) * 100
    return f"Trainable model parameters: {trainable_params}\nAll model parameters: {all_model_params}\nPercentage of trainable parameters: {percentage_trainable:.2f}%"


In [9]:
print(print_no_of_trainable_model_parameters(original_model))

Trainable model parameters: 247577856
All model parameters: 247577856
Percentage of trainable parameters: 100.00%


## Full fine tuning

In [12]:
def tokenize_func(example):
    start_prompt = "SUMMARIZE THE FOLLOWING CONVERSATION.\n\n"
    end_prompt = "\n\nSummary"
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    example['input_ids'] = tokenizer(prompt,padding ="max_length",truncation =True,return_tensors='pt').input_ids
    example['labels'] = tokenizer(example['summary'],padding='max_length',truncation=True,return_tensors='pt').input_ids

    return example
tokenized_datasets = dataset.map(tokenize_func,batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id','topic','dialogue','summary'])

Map: 100%|██████████| 1500/1500 [00:00<00:00, 3520.57 examples/s]
